# Examining TEMPO data retrieved from NASA Earthdata

## Overview

This notebook retrieves TEMPO data and inspects the characteristics of the data.

## Dataset Information

This notebook uses data from the [Tropospheric Emissions: Monitoring of Pollution (TEMPO)](https://asdc.larc.nasa.gov/project/TEMPO) instrument.

## Table of Contents

1. Setup
2. Search for data granules
3. Examining and downloading file results
4. Reading and inspecting the data
5. Working with the data to subset and plot
6. Using Harmony-py to pre-process and retrieve data
7. Plotting the data

### Notebook Author / Affiliation

Alexander Radkevich / Atmospheric Science Data Center (ASDC)

# 1. Setup

In [ ]:
import datetime as dt
import getpass
import shutil

import earthaccess  # needed to discover and download TEMPO data
import netCDF4 as nc  # needed to read TEMPO data
import numpy as np
import xarray as xr

import matplotlib.pyplot as plt  # needed to plot the resulting time series
import matplotlib.ticker as mticker
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

from harmony import BBox, Client, Collection, Request
from harmony.config import Environment

## Logging in

In [ ]:
# User needs to create an account at https://www.earthdata.nasa.gov/
# Function earthaccess.login prompts for EarthData login and password.
auth = earthaccess.login(strategy="interactive", persist=True)

# 2. Search for TEMPO granules

We are going to search for data about nitrogen dioxide ($NO_2$) concentrations from TEMPO.

In [ ]:
short_name = "TEMPO_NO2_L3"  # collection name to search for in the EarthData
version = "V03"

# Point of interest: NASA Langley Research Center, HamptonVA, USA
# latitude 37.1036 deg, longitude -76.3868 deg
# POI_lat = 37.1036
# POI_lon = -76.3868

# generic location, somewhere in the middle of the USA
POI_lat = 38.0
POI_lon = -96.0
date_start = "2024-09-01 00:00:00"
date_end = "2024-09-01 23:59:59"

POI_results = earthaccess.search_data(
    short_name=short_name,
    version=version,
    temporal=(date_start, date_end),
    point=(POI_lon, POI_lat),  # search by point of interest
)

dlat = 5.0  # deg
dlon = 6.0  # deg
bbox_results = earthaccess.search_data(
    short_name=short_name,
    version=version,
    temporal=(date_start, date_end),
    bounding_box=(
        POI_lon - dlon,
        POI_lat - dlat,
        POI_lon + dlon,
        POI_lat + dlat,
    ),  # search by bounding box
)

# 3. Examining and downloading file results

## What does a result look like?

In [ ]:
POI_results[0]

Let's get the link to a granule (`-1` to index the last granule in the results).

In [ ]:
print(POI_results[-1].data_links()[0])

Let's examine the file names present in the results object.

In [ ]:
for r in POI_results:
    granule_name = r.data_links()[0].split("/")[-1]
    print(granule_name)

## Downloading granulues

In [ ]:
files = earthaccess.download(POI_results[8:10], local_path=".")

# 4. Reading and inspecting the data

We first define a function for reading data from TEMPO_NO2_L3_V03 netCDF file.

In [ ]:
def read_TEMPO_NO2_L3(fn):
    with nc.Dataset(fn) as ds:  # open read access to file
        # Open group product, /product, as prod.
        prod = ds.groups["product"]

        # Read variable vertical_column_stratosphere from prod (group product, /product).
        var = prod.variables["vertical_column_stratosphere"]
        strat_NO2_column = var[:]  # create a numpy array
        fv_strat_NO2 = var.getncattr("_FillValue")

        # Read variable 'vertical_column_troposphere' from prod (group product, /product).
        var = prod.variables["vertical_column_troposphere"]
        trop_NO2_column = var[:]
        fv_trop_NO2 = var.getncattr("_FillValue")
        NO2_unit = var.getncattr("units")

        # Read variable 'main_data_quality_flag' from prod (group product, /product).
        QF = prod.variables["main_data_quality_flag"][:]

        # Read latitude and longitude variables from root (/) into a numpy array.
        lat = ds.variables["latitude"][:]
        lon = ds.variables["longitude"][:]

    return lat, lon, strat_NO2_column, fv_strat_NO2, trop_NO2_column, fv_trop_NO2, NO2_unit, QF

### Let's now examine the data in a granulue

In [ ]:
granule_name = POI_results[8].data_links()[0].split("/")[-1]
print(granule_name)
lat, lon, strat_NO2_column, fv_strat_NO2, trop_NO2_column, fv_trop_NO2, NO2_unit, QF = (
    read_TEMPO_NO2_L3(granule_name)
)

In [ ]:
print("unit of NO2 column is ", NO2_unit)  # unit of NO2 column

In [ ]:
# lat is a 1D array:
lat.shape

In [ ]:
# lat is a 1D array:
lon.shape

In [ ]:
# stratospheric NO2 column is a 3D array
# with second dimension being the number of latitudes and third being the number of longitudes:
strat_NO2_column.shape

In [ ]:
# and so is tropospheric NO2 column:
trop_NO2_column.shape

In [ ]:
fv_strat_NO2  # fill value for the array of stratospheric NO2 column

### How many valid (non-fill) values are there?

In [ ]:
len(strat_NO2_column[strat_NO2_column != fv_strat_NO2])

In [ ]:
len(trop_NO2_column[trop_NO2_column != fv_trop_NO2])

### How many pixels with high quality flag, 0, are there?

In [ ]:
len(QF[QF == 0])

### the numbers are different

### So, we need to create arrays of equal length to add them up in order to get total column

In [ ]:
good_data_mask = (QF == 0) & (trop_NO2_column != fv_trop_NO2) & (strat_NO2_column != fv_strat_NO2)
print(good_data_mask.shape)

### How many good pixels are there?

In [ ]:
good_trop_NO2_column = trop_NO2_column[good_data_mask]
len(good_trop_NO2_column)

### unfortunate reality - "good" pixels may contain negative column

In [ ]:
min(good_trop_NO2_column)

### getting physically meaningful pixels

In [ ]:
best_data_mask = good_data_mask & (trop_NO2_column > 0.0) & (strat_NO2_column > 0.0)

In [ ]:
best_trop_NO2_column = trop_NO2_column[best_data_mask]
len(best_trop_NO2_column)

# 5. Working with the data to subset and plot

### working with data: spatial subsetting and masking out bad data

In [ ]:
# Define a region of interest.
dlat = 5  # deg
dlon = 6  # deg
mask_lat = (lat > POI_lat - dlat) & (lat < POI_lat + dlat)
mask_lon = (lon > POI_lon - dlon) & (lon < POI_lon + dlon)

# Subset NO2 column arrays.
trop_NO2_column_loc = trop_NO2_column[0, mask_lat, :][:, mask_lon]
strat_NO2_column_loc = strat_NO2_column[0, mask_lat, :][:, mask_lon]
QF_loc = QF[0, mask_lat, :][:, mask_lon]
best_data_mask_loc = (QF_loc == 0) & (trop_NO2_column_loc > 0.0) & (strat_NO2_column_loc > 0.0)

# Create 2D arrays of latitudes and longitudes, by repeating lon/lat along rows/columns.
nlat, nlon = trop_NO2_column_loc.shape
lon_loc_2D = np.vstack([lon[mask_lon]] * nlat)
lat_loc_2D = np.column_stack([lat[mask_lat]] * nlon)

### plotting spatial distribution

In [ ]:
# Create a Cartopy projection
proj = ccrs.PlateCarree()
transform = ccrs.PlateCarree()

# Create a figure and axis.
fig, ax = plt.subplots(
    1, 1, figsize=(5, 6), dpi=300, facecolor=None, subplot_kw={"projection": proj}
)

# Add coastlines and U.S. state borders
ax.coastlines(linewidth=0.5)
ax.add_feature(cfeature.STATES, linestyle=":", edgecolor="gray", linewidth=0.5)

im = ax.scatter(
    lon_loc_2D[best_data_mask_loc],
    lat_loc_2D[best_data_mask_loc],
    s=1,
    c=trop_NO2_column_loc[best_data_mask_loc] + strat_NO2_column_loc[best_data_mask_loc],
    vmin=0,
    vmax=5.0e16,
    transform=transform,
)
ax.set_extent([-103, -89, 32, 44], crs=transform)

cb = plt.colorbar(im, ticks=[0, 1.0e16, 2.0e16, 3.0e16, 4.0e16, 5.0e16], fraction=0.022, pad=0.01)
cb.set_label("total NO2 col, " + NO2_unit, fontsize=10)

plt.show()

# 6. Another way to get the data - HarmonyPy

In [ ]:
print("Please provide your Earthdata Login credentials to allow data access")
print("Your credentials will only be passed to Earthdata and will not be exposed in the notebook")
username = input("Username:")

harmony_client = Client(env=Environment.PROD, auth=(username, getpass.getpass()))

### collection_ID is needed for this method
How do we get one?
Search for collection short name, TEMPO_NO2_L3, in EarthData https://search.earthdata.nasa.gov/search
and select right version, V03:
![img1](getting_collection_ID_1.jpg)
find collection_ID in the address line:
![img2](getting_collection_ID_2.jpg)

### getting specific datasets from a granule

In [ ]:
granule_name = POI_results[8].data_links()[0].split("/")[-1]

# TEMPO Formaldehyde L3 C2930763263-LARC_CLOUD
request = Request(
    collection=Collection(id="C2930763263-LARC_CLOUD"),
    granule_name=[granule_name],
    variables=[
        "product/vertical_column_troposphere",
        "product/vertical_column_stratosphere",
        "product/main_data_quality_flag",
        "latitude",
        "longitude",
    ],
)

job_id = harmony_client.submit(request)
print(f"jobID = {job_id}")
harmony_client.wait_for_processing(job_id, show_progress=True)

# Download the resulting files
results = harmony_client.download_all(job_id, directory=".", overwrite=True)
all_results_stored = [f.result() for f in results]
print(f"Number of result files: {len(all_results_stored)}")

### make sure we got the same numbers - read and test the subset

In [ ]:
granule_name = all_results_stored[0]
lat, lon, strat_NO2_column, fv_strat_NO2, trop_NO2_column, fv_trop_NO2, NO2_unit, QF = (
    read_TEMPO_NO2_L3(granule_name)
)

### How many pixels with valid column values and high quality flag?
there should be 7363186

In [ ]:
good_data_mask = (QF == 0) & (trop_NO2_column != fv_trop_NO2) & (strat_NO2_column != fv_strat_NO2)
len(trop_NO2_column[good_data_mask])

### copy subset file - it will be over-written at the next step

In [ ]:
for r in all_results_stored:
    print(r)
    ind = r.rfind(".")
    new_name = r[:ind] + "_variable_subset" + r[ind:]
    print(new_name)
    shutil.copy(r, new_name)

### spatial AND variable subset

In [ ]:
min_lat = POI_lat - dlat
max_lat = POI_lat + dlat
min_lon = POI_lon - dlon
max_lon = POI_lon + dlon

request = Request(
    collection=Collection(id="C2930763263-LARC_CLOUD"),
    # Note there is not a granule specified!
    spatial=BBox(min_lon, min_lat, max_lon, max_lat),
    temporal={
        "start": dt.datetime(2024, 9, 1, 0, 0, 0),
        "stop": dt.datetime(2024, 9, 1, 23, 59, 59),
    },
    variables=[
        "product/vertical_column_troposphere",
        "product/vertical_column_stratosphere",
        "product/main_data_quality_flag",
        "latitude",
        "longitude",
    ],
)

job_id = harmony_client.submit(request)

print(f"jobID = {job_id}")
harmony_client.wait_for_processing(job_id, show_progress=True)

# Download the resulting files
results = harmony_client.download_all(job_id, directory=".", overwrite=True)
all_results_stored = sorted([f.result() for f in results])
print(f"Number of result files: {len(all_results_stored)}")

### can the new subsetted granule be read by the our custom-made function read_TEMPO_NO2_L3(fn)?

In [ ]:
granule_name = all_results_stored[8]
print(granule_name)
lat, lon, strat_NO2_column, fv_strat_NO2, trop_NO2_column, fv_trop_NO2, NO2_unit, QF = (
    read_TEMPO_NO2_L3(granule_name)
)

### Yes, it can. But we can use another method to access netCDF files
using xarray library

In [ ]:
# Open the data files
ds_dict = dict()
for r in all_results_stored:
    ds_root = xr.open_dataset(r)
    ds_product = xr.open_dataset(r, group="product")

    ds_dict[r] = xr.merge([ds_root, ds_product])

# 7. Plotting the data

### plotting individual images

In [ ]:
for i, (dk, dv) in enumerate(ds_dict.items()):
    Var = dv["vertical_column_troposphere"].values
    QF = dv["main_data_quality_flag"].values
    best_trop_NO2_mask = (Var > 0.0) & (QF == 0)
    best_trop_NO2 = Var[best_trop_NO2_mask]

    (nt, nlat, nlon) = Var.shape
    lon_1D = dv["longitude"].values
    lon_2D = np.empty((nt, nlat, nlon))
    for j in range(nlat):
        lon_2D[0, j, :] = lon_1D
    lat_1D = dv["latitude"].values
    lat_2D = np.empty((nt, nlat, nlon))
    for j in range(nlon):
        lat_2D[0, :, j] = lat_1D
    print(i)

    # Create a figure
    fig = plt.figure(figsize=(5, 6), dpi=120, facecolor=None)
    # Create a Cartopy projection
    proj = ccrs.PlateCarree()
    transform = ccrs.PlateCarree()
    # Adding subplot
    ax = fig.add_subplot(111, projection=proj)
    # Add coastlines
    ax.coastlines(linewidth=0.5)
    # Add U.S. state borders
    ax.add_feature(cfeature.STATES, linestyle=":", edgecolor="gray", linewidth=0.5)
    # Set plot title, in this case - the name of subset granule
    ax.set_title(dk.split("/")[-1], fontsize=9)
    # Set map extent
    ax.set_extent([-103, -89, 32, 44], crs=transform)
    # Set gridlines
    gl = ax.gridlines(draw_labels=True, dms=True, color="gray", linestyle="--", linewidth=0.5)
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.top_labels = False  # no labels on the top margin
    gl.right_labels = False  # no labels on the right margin
    gl.xlines = True
    gl.ylines = True
    gl.xlocator = mticker.FixedLocator([-100, -96, -92])  # longitude ticks
    gl.ylocator = mticker.FixedLocator([34, 38, 42])  # latitude ticks

    if len(best_trop_NO2) == 0:
        im = ax.scatter([POI_lon], [POI_lat], s=0, c=[0], vmin=0, vmax=1.5e16, transform=transform)
    else:
        im = ax.scatter(
            lon_2D[best_trop_NO2_mask],
            lat_2D[best_trop_NO2_mask],
            s=1,
            c=best_trop_NO2,
            vmin=0,
            vmax=1.5e16,
            transform=transform,
        )

    cb = plt.colorbar(im, ticks=[0, 0.5e16, 1.0e16, 1.5e16], fraction=0.022, pad=0.1)
    cb.set_label("trop NO2 col, " + NO2_unit, fontsize=10)

    plt.show()

### plotting multi-panel image

In [ ]:
ngr = len(ds_dict)
# Create a figure
fig = plt.figure(figsize=(6, 5 * ngr), dpi=120, facecolor=None)
# Create a Cartopy projection
proj = ccrs.PlateCarree()
transform = ccrs.PlateCarree()

for i, (dk, dv) in enumerate(ds_dict.items()):
    Var = dv["vertical_column_troposphere"].values
    QF = dv["main_data_quality_flag"].values
    best_trop_NO2_mask = (Var > 0.0) & (QF == 0)
    best_trop_NO2 = Var[best_trop_NO2_mask]

    (nt, nlat, nlon) = Var.shape
    lon_1D = dv["longitude"].values
    lon_2D = np.empty((nt, nlat, nlon))
    for j in range(nlat):
        lon_2D[0, j, :] = lon_1D
    lat_1D = dv["latitude"].values
    lat_2D = np.empty((nt, nlat, nlon))
    for j in range(nlon):
        lat_2D[0, :, j] = lat_1D
    print(i)

    # Adding subplot
    ax = fig.add_subplot(ngr, 1, i + 1, projection=proj)
    # Add coastlines
    ax.coastlines(linewidth=0.5)
    # Add U.S. state borders
    ax.add_feature(cfeature.STATES, linestyle=":", edgecolor="gray", linewidth=0.5)
    # Set plot title, in this case - the name of subset granule
    ax.set_title(dk.split("/")[-1], fontsize=9)
    # Set map extent
    ax.set_extent([-103, -89, 32, 44], crs=transform)
    # Set gridlines
    gl = ax.gridlines(draw_labels=True, dms=True, color="gray", linestyle="--", linewidth=0.5)
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.top_labels = False  # no labels on the top margin
    gl.right_labels = False  # no labels on the right margin
    gl.xlines = True
    gl.ylines = True
    gl.xlocator = mticker.FixedLocator([-100, -96, -92])  # longitude ticks
    gl.ylocator = mticker.FixedLocator([34, 38, 42])  # latitude ticks

    if len(best_trop_NO2) == 0:  # if there are no data points, ...
        im = ax.scatter(
            [POI_lon], [POI_lat], s=0, c=[0], vmin=0, vmax=1.5e16, transform=transform
        )  # ... plot an empty image
    else:
        im = ax.scatter(
            lon_2D[best_trop_NO2_mask],
            lat_2D[best_trop_NO2_mask],
            s=1,
            c=best_trop_NO2,
            vmin=0,
            vmax=1.5e16,
            transform=transform,
        )

    cb = plt.colorbar(im, ticks=[0, 0.5e16, 1.0e16, 1.5e16], fraction=0.022, pad=0.1)
    cb.set_label("trop NO2 col, " + NO2_unit, fontsize=10)

plt.show()